<a href="https://colab.research.google.com/github/Twinkle-gawri/Word2Vec/blob/main/Word2Vec_Gensim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gensim

  Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
  Using cached scipy-1.13.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 8.0 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc 5.21.2 requires numpy>=1.25.0, but you have numpy 1.24.3 which is incompatible.
albumentations 2.0.5 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have 

In [1]:
from gensim import downloader

In [3]:
wv = downloader.load('glove-wiki-gigaword-50')

[==================================================] 100.0% 66.0/66.0MB downloaded


In [4]:
wv.most_similar('car',topn=5)

[('truck', 0.92085862159729),
 ('cars', 0.8870189785957336),
 ('vehicle', 0.8833683729171753),
 ('driver', 0.8464019298553467),
 ('driving', 0.8384189009666443)]

In [5]:
wv.most_similar(positive=['car','minivan']) # It finds words that are similar to the combination of 'car' and 'minivan'.

[('truck', 0.9100273251533508),
 ('suv', 0.904007613658905),
 ('jeep', 0.8619828820228577),
 ('vehicle', 0.8431736826896667),
 ('cars', 0.8421834111213684),
 ('parked', 0.8129159212112427),
 ('dealership', 0.7910657525062561),
 ('pickup', 0.7901358604431152),
 ('mercedes', 0.787341296672821),
 ('wagon', 0.7841063737869263)]

In [7]:
print(wv.most_similar(positive=['woman','king'],negative=['man'],topn=1))

"""
positive=['woman', 'king'] -> Add the vector for woman and king.
negative=['man'] -> Subtract the vector for man.
topn=1 -> Return the single most similar word to the result.
"""

[('queen', 0.8523604273796082)]


"\npositive=['woman', 'king'] -> Add the vector for woman and king.\nnegative=['man'] -> Subtract the vector for man.\ntopn=1 -> Return the single most similar word to the result.\n"

In [8]:
wv.similarity('man','woman')

0.8860338

In [9]:
wv.doesnt_match(['land','air','sea','water','car'])  # Find which word doesn't fit (i.e., is the least similar) among the given list.

'car'

In [10]:
# This code initializes the model with those parameters, but does not train it yet.
from gensim.models import Word2Vec
model = Word2Vec(window=2, # sets the size of the context window to 2.
         negative=20, # number of negative samples used in the negative sampling process.
         min_count=5, # Ignores all words with total frequency lower than 5 in the corpus.
         vector_size=300) # Each word will be represented by a 300-dimensional vector.

When training a Word2Vec model, you can't just jump into training. First, the model needs to:
* Scan your sentences
* Count word frequencies
* Filter words based on min_count
* Create a vocabulary
    * Map words to unique IDs
    * Prepare internal tables needed for training (like for negative sampling)

In [11]:
import pandas as pd
data=pd.read_csv('/content/train.csv')

In [12]:
data.dropna(axis=0, how='any', inplace=True)
"""
This removes any rows from the DataFrame data that contain at least one missing (NaN) value.
  axis=0 --> specifies rows.
  how='any' --> drop the row if any column is NaN.
  inplace=True --> the changes are made directly to the original DataFrame (data), not returning a new one.
"""
sentences = [row.split() for row in data['text']]
"""
This creates a list of tokenized sentences from the 'text' column of your DataFrame.
Each row in data['text'] is assumed to be a string (e.g., a sentence or paragraph).
.split() splits each string into a list of words (tokens), usually based on whitespace.
"""

"\nThis creates a list of tokenized sentences from the 'text' column of your DataFrame.\nEach row in data['text'] is assumed to be a string (e.g., a sentence or paragraph).\n.split() splits each string into a list of words (tokens), usually based on whitespace.\n"

In [13]:
model.build_vocab(sentences,progress_per=1000)
"""
Scans your tokenized sentences (sentences) to build the vocabulary.
progress_per=1000 = it lets you see that the process is working and gives you a sense of progress.
"""

In [14]:
model.train(sentences, epochs=30, total_examples=model.corpus_count) # Actual Training Phase , (trained_word_count, total_word_count)

(6843056, 10637130)

In [15]:
model.wv['sad']

array([ 3.89738023e-01,  4.05321196e-02, -9.46059704e-01, -5.56559682e-01,
        2.38109902e-01, -1.02653956e+00,  4.80230629e-01,  6.24811471e-01,
       -9.29139927e-02, -2.40441173e-01, -1.12344384e-01,  4.16138828e-01,
        3.45952839e-01,  9.92363393e-01,  8.63126814e-02, -1.00347555e+00,
        3.11229676e-01,  6.21255279e-01, -3.48913148e-02, -1.66044444e-01,
        6.80994034e-01,  2.08367616e-01,  2.99134314e-01, -6.39072657e-01,
       -2.57314891e-01,  1.29199350e+00, -7.32435167e-01, -5.09418488e-01,
        5.91552794e-01,  2.06318304e-01, -1.18690050e+00, -4.07797366e-01,
       -9.18258190e-01,  1.24120913e-01, -5.19405603e-01,  8.19421113e-02,
       -1.04695596e-01, -1.07615851e-01, -3.03642303e-01,  3.81280243e-01,
       -1.21719040e-01,  1.48522854e-01, -8.38146091e-01, -8.65654230e-01,
        8.15935910e-01,  1.23482645e+00, -2.91331738e-01, -8.24882567e-01,
       -8.16204429e-01,  1.04357339e-01,  7.87321806e-01, -1.16564892e-02,
        3.76984984e-01,  

In [16]:
model.wv.most_similar('sad')

[('bummed', 0.5891948938369751),
 ('disappointed', 0.5763925313949585),
 ('sad.', 0.558874785900116),
 ('sad!', 0.5587061047554016),
 ('excited', 0.5293232202529907),
 ('depressed', 0.5233137011528015),
 ('upset', 0.5128488540649414),
 ('nervous', 0.5039489269256592),
 ('sad...', 0.49715086817741394),
 ('funny,', 0.4956815540790558)]